In [66]:
# pip install GitPython
# pip3 install BeautifulSoup4

In [129]:
import os 
import openai
import pprint
import shutil
import requests
from git import Repo
from pathlib import Path
from bs4 import BeautifulSoup as BSoup
pp = pprint.PrettyPrinter(depth=100)

In [140]:
# git hub pages path
# https://scottyadean.github.io/openai-auto-blog/

In [113]:
OPEN_AI_KEY = os.environ["OPENAI_KEY"]
openai.api_key = OPEN_AI_KEY

In [68]:
pwd

'C:\\Users\\scott\\apps\\openai\\notebooks'

In [69]:
PATH_TO_REPO = Path('C:\\Users\\scott\\apps\\openai\\notebooks\\openai-auto-blog\\.git')
print(PATH_TO_REPO.parent)
PATH_TO_INDEX = PATH_TO_REPO.parent
PATH_TO_ASSETS = PATH_TO_INDEX/"assets"
PATH_TO_CONTENT = PATH_TO_INDEX/"content"

print(PATH_TO_CONTENT)

C:\Users\scott\apps\openai\notebooks\openai-auto-blog
C:\Users\scott\apps\openai\notebooks\openai-auto-blog\content


In [141]:
def push_blog_post(commit_msg="Update Blog Post"):
    """ Push the content of the blog to git hub pages """
    repo = Repo(PATH_TO_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_msg)
    origin = repo.remote(name='origin')
    origin.push()
    
def create_blog_post(title, content, cover_img):
    """ This takes the content from openai and injects it into a html page """
    img = Path(cover_img)
    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    name = f'{files+1}.html'
    path = PATH_TO_CONTENT/name
    shutil.copy( img, PATH_TO_ASSETS )

    if not os.path.exists(path):
        with open(path, 'w') as _f:
            _f.write(f"<!doctype html><html></head><title>{title}</title></head><body><section>")
            _f.write(f"<img src='../assets/{img.name}' alt='{title}' />")
            _f.write(f"<h1>{title}</h1>")
            _f.write(f"<div class='content'>")
            _f.write(content.replace("\n", "<br />\n"))
            _f.write(f"</div>")
            _f.write(f"</section></body></html>")   
    else:    
        raise FileExistsError('File Already Exists')
    return name

def add_blog_link(path, href, name):
    """ This adds a link to the blog post to the index.html page.  """
    with open(path, 'r') as html:
        soup = BSoup(html.read())
        atag = soup.new_tag("a", href=href )
        atag.string = name
        soup.html.body.section.div.append(atag)

    with open(path, 'w') as _f:
        _f.write(str( soup.prettify( formatter='html') ) )

def create_prompt(title, cats = "tech, python, coding, AI"):
    """ this creates a prompt for openai to better create the text compleation request """
    prompt = f"""
        Biography:
        My name is Scott Dean and I am a Software Engineer. 
        Blog
        Title: {title}
        tags: {cats}
        Full Text:"""
    return prompt

def ask_open_ai(prompt):
    """ send the prompt to the openai """
    res = openai.Completion.create(model="text-davinci-003",
                                   prompt = prompt,
                                   temperature = 0.7,
                                   max_tokens = 1000)
    return res

def get_img(title):
    """ Generate an image for the blog post based on the title.  """
    prompt = f"{title}"
    img = openai.Image.create( prompt=prompt, n=1, size='512x512' )
    return img

def save_img(url, file_output_name):
    """ save the image locally that openai created """
    img = requests.get(url, stream=True)
    if img.status_code == 200:
        with open(file_output_name, 'wb') as _f:
            shutil.copyfileobj(img.raw, _f)
    else:
        print("unable to download file")
    return img.status_code


def publish_blog_post(title, img_type="", cats="Blog, Personal, Web Publishing"):
    """ create the blog post and push it to git hub pages.  """
    blog = ask_open_ai(create_prompt(title, cats))
    content = blog['choices'][0]['text']
    img = get_img(f"{img_type}{title}")
    url = f'{title.replace(" ", "-").lower()}.png'
    save_img(img['data'][0]['url'], url)
    file_name = create_blog_post( title, content, url  )
    add_blog_link(PATH_TO_INDEX/"index.html", f"./content/{file_name}", title)
    push_blog_post(f"Added: {title}")
    print("Complete ")



In [122]:
## create new html page with blog conent
# create_blog_post( "new blog", "This is the body content\n and some subtext", 'logo.png'  )
## example push update to git repo
# push_blog_post("create content test commit 5")
## BSoup add new link to indx 
# add_blog_link(PATH_TO_INDEX/"index.html", "./content/1.html", "sample link")

In [123]:
## Ask open ai to create blog content

In [125]:
## testing prompt output
blog = ask_open_ai(create_prompt("The future of python and AI"))

In [127]:
content = blog['choices'][0]['text']

In [128]:
content

'\n\nAs technology advances, so does the potential for Artificial Intelligence (AI). Python has long been a leader in the world of programming languages, and its potential to be used for AI is no different. Python is a popular choice for developers looking to create powerful AI-driven applications.\n\nAI is becoming increasingly advanced and can now be used for a range of applications. From natural language processing to predictive analytics, AI can be used to solve complex problems and generate valuable insights from data. Python is an ideal language for AI development due to its versatility and ease of use.\n\nPython libraries such as Scikit-learn, TensorFlow, and Keras make it easy to build powerful AI applications. These libraries allow developers to create powerful neural networks that can process data and generate insights.\n\nPython has become an increasingly popular language for AI development due to its versatility and wide range of applications. With its ease of use and power

In [134]:
# Testing saving image. 
title = "The future of python and AI"
img = get_img(f"An Oil Painting Of {title}")
url = f'{title.replace(" ", "-").lower()}.png'
save_img(img['data'][0]['url'], url)

200

In [135]:
create_blog_post( title, content, url  )

WindowsPath('C:/Users/scott/apps/openai/notebooks/openai-auto-blog/content/2.html')

In [136]:
# testing adding link to index. 
add_blog_link(PATH_TO_INDEX/"index.html", "./content/2.html", title)
push_blog_post(f"Added: {title}")

In [142]:

## invoke the flow 
publish_blog_post( "3 of the best pizza restaurants to eat in New Youk City before you die", "Professional food Photography, 15mm studio lighting" "Food, Pizza, Travel, New York" )




Complete 
